# Binocular

We explore a different approach of training per patient by loading both left and right image and train efficient net b0 

We also treat ODIR as multi-label problem instead of multi-class as originally it is officially a multi-label problem
from https://odir2019.grand-challenge.org/dataset/
> Note: one patient may contains one or multiple labels. 

We also want to explore binocular or siamese approach to train our model on both left and right fundus image pair. This has been researched in https://arxiv.org/html/2504.18046v3 DMS-Net:Dual-Modal Multi-Scale Siamese Network for Binocular: Fundus Image Classification Guohao Huo, Zibo Lin, Zitong Wang, Ruiting Dai, Hao Tang paper to work well for fundus disease classification 

There are 3 advantages of use both eyes images instead of one eye image :
- Symmetry: Diseases like Diabetes aren't "accidents" in one eye; they are systemic. If the AI sees it in both, it's a "confirmed" diagnosis.

- Comparison: The left eye acts as a "control" for the right eye. AI can spot a tiny change by noticing how much it differs from the other eye.

- Noise Reduction: Just like your two eyes help you see depth, two images help the AI ignore "camera blur" or "dust" on one lens that might look like a disease.

Install Dependencies

In [1]:
%%capture
!pip install -q kagglehub torch torchvision scikit-learn pandas opencv-python tqdm wandb

Import python libraries

In [2]:

import os
import cv2
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, multilabel_confusion_matrix, accuracy_score
import kagglehub
from tqdm import tqdm # tqdm for progress bars
import wandb


Download DataSet

In [3]:
# 1. Download Dataset (Official ODIR-5K)
path = kagglehub.dataset_download("andrewmvd/ocular-disease-recognition-odir5k")
print("Dataset path:", path)
IMG_DIR = os.path.join(path, "ODIR-5K/ODIR-5K/Training Images")
CSV_PATH = os.path.join(path, "full_df.csv")
TRAIN_CSV_PATH = "train.csv"
VAL_CSV_PATH =  "val.csv"
TEST_CSV_PATH =  "test.csv"
IMG_SIZE = 512
BATCH_SIZE = 4
ACCUMULATION_STEPS = 8
EPOCHS = 30
PATIENCE = 5 # Early stopping patience in epochs, stop if no improvement in F1 score for this many epochs
LEARNING_RATE = 1e-4
NUM_CLASSES = 8
NUM_WORKERS = 2
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
df = pd.read_csv(CSV_PATH)
train_df = pd.read_csv(TRAIN_CSV_PATH)
val_df = pd.read_csv(VAL_CSV_PATH)
test_df = pd.read_csv(TEST_CSV_PATH)
FAST_IMG_DIR = f"tmp/processed_{IMG_SIZE}_images"
os.makedirs(FAST_IMG_DIR, exist_ok=True)
RUN_NAME = f"efficient-b4_bbgsa_{IMG_SIZE}" # Unique name for this run, used for saving models and logging
SAVED_MODELS_DIR = "saved_models"
os.makedirs(SAVED_MODELS_DIR, exist_ok=True)
SAVED_MODEL_PATH = os.path.join(SAVED_MODELS_DIR, f"{RUN_NAME}_best.pth")
CHECKPOINT_PATH = os.path.join(SAVED_MODELS_DIR, f"{RUN_NAME}_checkpoint.pth")
CLASS_NAMES = ['Normal', 'Diabetes', 'Glaucoma', 'Cataract', 'AMD', 'Hypertension', 'Myopia', 'Other']
CLASS_CODES = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']


Dataset path: /home/ray/.cache/kagglehub/datasets/andrewmvd/ocular-disease-recognition-odir5k/versions/2


In [4]:
wandb.init(project="odir-2019", name=RUN_NAME, config={
    "img_size": IMG_SIZE, "lr": 1e-4, 
    "batch_size": BATCH_SIZE, "accumulation_steps": ACCUMULATION_STEPS, 
    "epochs": EPOCHS, "patience": PATIENCE})

wandb: Currently logged in as: raymond-samalo (samalo) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Ben Graham's Preprocessing

This function implements the Ben Graham Preprocessing 
ref : 
- https://scholar.google.com/citations?view_op=view_citation&hl=en&user=jQkkhlkAAAAJ&citation_for_view=jQkkhlkAAAAJ:sNmaIFBj_lkC
- https://scholar.google.com/citations?user=jQkkhlkAAAAJ&hl=en


From https://medium.com/@astronomer.abdurrehman/enhancing-image-quality-for-machine-learning-ben-grahams-preprocessing-e795ad982abe
the method described as followed
<blockquote>

The cv2.GauissanBlur takes an image, (0, 0) tuple automatically chooses a gaussian filter size based on sigmaX value which specifies the intensity of blur. Goal of using gaussian blur here is to reduce the noise and smooth out the fine details.

The addWeighted function blends two images together using specified weights, the -4 here is the beta value which subtracts the blurred image from the original image and 128 is the gamma value that adjusts the brightness so that the image does not become too dark after subtraction.


</blockquote>


In [5]:
def ben_graham_prep(img, sigmaX=10):
    """Enhances vessels and normalizes lighting."""
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Circular Crop: Find non-black pixels and crop
    mask = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) > 10
    if np.any(mask):
        coords = np.argwhere(mask)
        y0, x0 = coords.min(axis=0)
        y1, x1 = coords.max(axis=0)
        img = img[y0:y1, x0:x1]
    
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    blurred = cv2.GaussianBlur(img, (0, 0), sigmaX)
    enhanced = cv2.addWeighted(img, 4, blurred, -4, 128)
    return enhanced

On the fly image prep caused the training slowdown given the image need to be preprocessed repeatedly each time it is loaded. We speed up the process by performing preprocessing offline once and cache it

In [6]:
def run_offline_prep(df, raw_dir, img_prep_func, save_dir):
    print("🚀 Starting Offline Pre-processing (Ben Graham)...")
    all_images = pd.concat([df['Left-Fundus'], df['Right-Fundus']]).unique()
    for img_name in tqdm(all_images):
        save_path = os.path.join(save_dir, img_name)
        if not os.path.exists(save_path):
            img = cv2.imread(os.path.join(raw_dir, img_name))
            # Ben Graham Logic
            enhanced = img_prep_func(img)
            cv2.imwrite(save_path, cv2.cvtColor(enhanced, cv2.COLOR_RGB2BGR))

In [7]:
run_offline_prep(df, IMG_DIR, ben_graham_prep, FAST_IMG_DIR)

🚀 Starting Offline Pre-processing (Ben Graham)...


100%|██████████| 6716/6716 [00:00<00:00, 412562.55it/s]


## Data Loader 

Previously we load the data and then performed preprocessing on the fly
Given we did the preprocessing offline, we can now simply load the image 

In [8]:
class FastODIRDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        # Target classes: Normal, Diabetes, Glaucoma, Cataract, AMD, Hypertension, Myopia, Other
        self.labels = df[['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        l_img_path = os.path.join(self.img_dir, row['Left-Fundus'])
        r_img_path = os.path.join(self.img_dir, row['Right-Fundus'])
        
        # Load and Preprocess
        l_img = cv2.cvtColor(cv2.imread(l_img_path), cv2.COLOR_BGR2RGB)
        r_img = cv2.cvtColor(cv2.imread(r_img_path), cv2.COLOR_BGR2RGB )
        
        if self.transform:
            l_img = self.transform(l_img)
            r_img = self.transform(r_img)
            
        return l_img, r_img, torch.tensor(self.labels[idx], dtype=torch.float32)

In [9]:
class ODIRDualNet(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES):
        super().__init__()
        # Using B0 for efficiency, upgrade to B4 for better accuracy
        self.backbone = models.efficientnet_b4(weights='DEFAULT') # use pretrained weights for better feature extraction
        # ref https://docs.pytorch.org/vision/main/models/generated/torchvision.models.efficientnet_b0.html#torchvision.models.EfficientNet_B0_Weights
        self.feature_dim = self.backbone.classifier[1].in_features # Get feature dimension before classifier
        self.backbone.classifier = nn.Identity() # Remove top layer

        self.classifier = nn.Sequential( # replace classifier with a custom head that combines features from both eyes
            nn.Linear(self.feature_dim * 2, IMG_SIZE), # Combine features from both eyes
            nn.ReLU(), # Non-linearity for better learning relu f(x) = max(0, x)
            nn.Dropout(0.3), # Regularization to prevent overfitting
            nn.Linear(IMG_SIZE, num_classes) # Final output layer for multi-label classification
        )

    def forward(self, left, right):
        l_feat = self.backbone(left) # Extract features from left eye
        r_feat = self.backbone(right) # Extract features from right eye
        combined = torch.cat((l_feat, r_feat), dim=1) # Combine features from both eyes
        return self.classifier(combined) # Pass through classifier to get final predictions

## Thresholds

Instead of using 1 thresholds 0.5 for all labels, we find the best or optimise threshold for each label individually to maximise our F1-score

In [10]:
def find_best_thresholds(y_true, y_probs):
    thresholds = np.linspace(0.1, 0.9, 81) # Test thresholds from 0.1 to 0.9 with fine granularity
    best_ts = np.zeros(NUM_CLASSES) 
    for i in range(NUM_CLASSES):
        best_f1 = 0 # Initialize best F1 score for this class
        for t in thresholds: # Test each threshold and calculate F1 score 
            score = f1_score(y_true[:, i], (y_probs[:, i] > t).astype(int), zero_division=0) # zero_division=0 to handle cases where there are no positive predictions
            if score > best_f1: 
                best_f1 = score # Update best F1 score for this class
                best_ts[i] = t # Update best threshold for this class
    return best_ts # Return array of best thresholds for each class

Data Loader with ImageNet Transformation


"All pre-trained models expect input images normalized in the same way, i.e. mini-batches of 3-channel RGB images of shape (3 x H x W), where H and W are expected to be at least 224. The images have to be loaded in to a range of [0, 1] and then normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225]."
Ref:
- https://docs.pytorch.org/vision/0.9/models.html.

model definition and training

In [11]:
from torch import autocast
from torch.amp.grad_scaler import  GradScaler 

# val and test transforms (no augmentation, just normalization)
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
# Data augmentation for training set 
train_transform = transforms.Compose([
        transforms.ToPILImage(),
        # Note: Horizontal/Vertical flips may not be appropriate for medical images as they can change the anatomical orientation. Use with caution.
        # hence commented out for now, can be enabled if you want to experiment with it
        #transforms.RandomHorizontalFlip(p=0.5),
        #transforms.RandomVerticalFlip(p=0.5),
        # Rotation and Color Jitter can help the model generalize better by simulating real-world variations in the images
        transforms.RandomRotation(degrees=20),
        transforms.ColorJitter(brightness=0.1, contrast=0.1),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

train_loader = DataLoader(FastODIRDataset(train_df, FAST_IMG_DIR, train_transform), 
                            batch_size=BATCH_SIZE, 
                            shuffle=True,
                            num_workers=NUM_WORKERS,
                            pin_memory=True)
val_loader = DataLoader(FastODIRDataset(val_df, FAST_IMG_DIR, transform), 
                        batch_size=BATCH_SIZE,
                        num_workers=NUM_WORKERS, 
                        pin_memory=True)
test_loader = DataLoader(FastODIRDataset(test_df, FAST_IMG_DIR, transform), 
                         batch_size=BATCH_SIZE,
                         num_workers=NUM_WORKERS, 
                         pin_memory=True)
model = ODIRDualNet().to(DEVICE)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)
scaler = GradScaler(DEVICE)
# ⚡ ACCELERATION: Compile the model (Requires PyTorch 2.0+)
# This can provide a 10-20% speedup in training time
if hasattr(torch, 'compile'):
    model = torch.compile(model)
    print("✅ Model Compiled for speed.")

start_epoch, best_f1, counter = 0, 0, 0

if os.path.exists(CHECKPOINT_PATH):
    checkpoint = torch.load(CHECKPOINT_PATH)
    model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scaler.load_state_dict(checkpoint['scaler_state_dict'])
    best_ts = checkpoint['thresholds']
    counter = checkpoint.get('counter', 0)
    start_epoch, best_f1, counter = checkpoint['epoch'] + 1, checkpoint['best_f1'], checkpoint.get('counter', 0)
    print(f"Resuming from epoch {start_epoch}")

for epoch in range(start_epoch, EPOCHS):
        model.train()
        train_loss = 0
        tr_preds, tr_true = [], []
        optimizer.zero_grad()

        for i, (l, r, y) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}")):
            l, r, y = l.to(DEVICE), r.to(DEVICE), y.to(DEVICE)
            
            with autocast(device_type=DEVICE):
                preds = model(l, r) # logits output from the model
                loss = criterion(preds, y) / ACCUMULATION_STEPS
            
            scaler.scale(loss).backward()
            
            if (i + 1) % ACCUMULATION_STEPS == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
            
            train_loss += loss.item() * ACCUMULATION_STEPS
            tr_preds.append(torch.sigmoid(preds).detach().cpu().numpy())
            tr_true.append(y.cpu().numpy())
        # --- Validation ---
        model.eval()
        val_preds, val_true = [], []
        with torch.no_grad():
            for l, r, y in val_loader:
                out = torch.sigmoid(model(l.to(DEVICE), r.to(DEVICE)))
                val_preds.append(out.cpu().numpy())
                val_true.append(y.numpy())
        
        val_probs = np.vstack(val_preds)
        val_true = np.vstack(val_true)
        best_ts = find_best_thresholds(val_true, val_probs)
        tr_probs, tr_true = np.vstack(tr_preds), np.vstack(tr_true)
        # Calculate Macro F1 with optimized thresholds
        val_preds_binary = (val_probs > best_ts).astype(int)
        tr_preds_binary = (tr_probs > best_ts).astype(int)
        val_f1 = f1_score(val_true, val_preds_binary, average='macro')
        train_f1 = f1_score(tr_true, tr_preds_binary, average='macro')
        train_acc = accuracy_score(tr_true, tr_preds_binary)
        val_acc = accuracy_score(val_true, val_preds_binary)
        # NEW: Step the scheduler based on Validation F1
        scheduler.step(val_f1)
        current_lr = optimizer.param_groups[0]['lr']

        # Log per-class F1 for visibility
        per_class_f1 = f1_score(val_true, (val_probs > best_ts).astype(int), average=None)
        metrics_dict = {f"val_f1_{name}": f for name, f in zip(CLASS_NAMES, per_class_f1)}
        metrics_dict.update({"epoch": epoch+1, "val_f1": val_f1, "train_f1": train_f1, "lr": current_lr, "train_loss": train_loss / len(train_loader),
                             "train_acc": train_acc, "val_acc": val_acc})
        wandb.log(metrics_dict)
        print(f"Epoch {epoch+1} | Loss: {train_loss/len(train_loader):.4f} | train_f1: {train_f1:.4f} | val_f1: {val_f1:.4f}")
        if val_f1 > best_f1:
            best_f1 = val_f1
            counter = 0 # reset counter on improvement
            torch.save({'model': model.state_dict(), 'thresholds': best_ts}, SAVED_MODEL_PATH)
            print(f"🚀 New Best Model Saved! F1: {val_f1:.4f}")
        # Save checkpoint every 5 epochs or if no improvement for 3 epochs
        if (epoch + 1) % 5 == 0:
            torch.save({
                'epoch': epoch,
                'model': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scaler_state_dict': scaler.state_dict(),
                'best_f1': best_f1,
                'counter': counter,
                'thresholds': best_ts
            }, CHECKPOINT_PATH)
            print(f"💾 Checkpoint saved at epoch {epoch+1}")
        if  PATIENCE>0 and counter >= PATIENCE: # early stopping if no improvement for 3 epochs
            print("⏹️ Early stopping triggered.")
            break

Downloading: "https://download.pytorch.org/models/efficientnet_b4_rwightman-23ab8bcd.pth" to /home/ray/.cache/torch/hub/checkpoints/efficientnet_b4_rwightman-23ab8bcd.pth


100%|██████████| 74.5M/74.5M [00:07<00:00, 9.80MB/s]


AcceleratorError: CUDA error: CUDA-capable device(s) is/are busy or unavailable
Search for `cudaErrorDevicesUnavailable' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


## Test Evaluation

In [ ]:
best_model = torch.load(SAVED_MODEL_PATH, weights_only=False)
thresholds = best_model['thresholds']
model.load_state_dict(best_model['model'])
print("Label Thresholds and Macro F1 Score of the Best Model:")
print("Labels: Normal, Diabetes, Glaucoma, Cataract, AMD, Hypertension, Myopia, Other")
print("Best Thresholds:", thresholds)
print("Best Macro F1 Score:", best_f1)
print("✅ Training Complete. Best model and thresholds saved.")

In [ ]:
model.eval()
t_probs, t_true = [], []
with torch.no_grad():
    for l, r, y in test_loader:
        out = torch.sigmoid(model(l.to(DEVICE), r.to(DEVICE)))
        t_probs.append(out.cpu().numpy()); t_true.append(y.numpy())

t_p, t_t = np.vstack(t_probs), np.vstack(t_true)
t_preds = (t_p > thresholds).astype(int)

mcm = multilabel_confusion_matrix(t_t, t_preds)
print("Multilabel Confusion Matrix:")
for i, class_name in enumerate(CLASS_NAMES):
    tn, fp, fn, tp = mcm[i].ravel()
    print(f"{class_name}: TP={tp}, FP={fp}, TN={tn}, FN={fn}")
    wandb.log({f"CM_{class_name}": wandb.plot.confusion_matrix(
            probs=None,
            y_true=t_t[:, i],
            preds=t_preds[:, i],
            title=f"Confusion Matrix for {class_name}",
            class_names=["Absent", "Present"]
        )})
print("\nClassification Report:")
print(classification_report(t_t, t_preds, target_names=CLASS_NAMES))
wandb.finish()